In [3]:
from google.colab import files
uploaded = files.upload()

Saving renttherunway_final_data.json.gz to renttherunway_final_data.json.gz


In [1]:
import gzip
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np

In [4]:
df = pd.read_json("renttherunway_final_data.json.gz",lines = True)
df = shuffle(df)

In [5]:
bert_df = df[["review_text","review_summary","rating"]]
bert_df["review_total"] = bert_df["review_summary"]+ " " +bert_df["review_text"]

bert_df=bert_df.drop(["review_text","review_summary"],axis=1)
bert_df = bert_df[:170000]

def convert_rating(rating):
    if rating > 8:
        out = "positive"
    elif rating == 8:
        out = "neutral"
    else:
        out = "negative"
    return out

bert_df["target"] = bert_df["rating"].apply(lambda x: convert_rating(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:

vocab_size = 5000 # make the top list of words (common words)
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary
training_portion = 0.9

In [9]:
bert_df.head()

,rating,review_total,target
114629,10.0,Loved this dress! Color is a beautiful vibrant...,positive
26414,8.0,Great material and fit for an August wedding ...,neutral
77015,10.0,Pretty dress and great customer service! Beau...,positive
187293,10.0,"Beautiful dress that fit perfectly, was comfor...",positive
150104,4.0,"Had high hopes, but this dress didn't work for...",negative


In [10]:
articles = []
labels = []
for index, row in bert_df.iterrows():
  labels.append(row['target'])
  article = row['review_total']
  for word in STOPWORDS:
    token = ' ' + word + ' '
    article = article.replace(token, ' ')
    article = article.replace(' ', ' ')
  articles.append(article)

In [11]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

In [12]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

In [13]:
train_sequences = tokenizer.texts_to_sequences(train_articles)

In [14]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [15]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [16]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [17]:
label_tokenizer.word_index

{'positive': 1, 'neutral': 2, 'negative': 3}

In [18]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(4,activation='softmax'))

In [19]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [21]:
num_epochs = 8
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Epoch 1/8
4782/4782 - 78s - loss: 0.5499 - accuracy: 0.7572 - val_loss: 0.5725 - val_accuracy: 0.7421 - 78s/epoch - 16ms/step
Epoch 2/8
4782/4782 - 78s - loss: 0.5397 - accuracy: 0.7626 - val_loss: 0.5734 - val_accuracy: 0.7421 - 78s/epoch - 16ms/step
Epoch 3/8
4782/4782 - 77s - loss: 0.5296 - accuracy: 0.7684 - val_loss: 0.5785 - val_accuracy: 0.7429 - 77s/epoch - 16ms/step
Epoch 4/8
4782/4782 - 78s - loss: 0.5226 - accuracy: 0.7718 - val_loss: 0.5872 - val_accuracy: 0.7416 - 78s/epoch - 16ms/step
Epoch 5/8
4782/4782 - 77s - loss: 0.5155 - accuracy: 0.7744 - val_loss: 0.5837 - val_accuracy: 0.7389 - 77s/epoch - 16ms/step
Epoch 6/8
4782/4782 - 78s - loss: 0.5083 - accuracy: 0.7792 - val_loss: 0.6012 - val_accuracy: 0.7408 - 78s/epoch - 16ms/step
Epoch 7/8
4782/4782 - 77s - loss: 0.5029 - accuracy: 0.7815 - val_loss: 0.5909 - val_accuracy: 0.7377 - 77s/epoch - 16ms/step
Epoch 8/8
4782/4782 - 78s - loss: 0.4976 - accuracy: 0.7837 - val_loss: 0.5974 - val_accuracy: 0.7388 - 78s/epoch - 16

In [24]:
txt = ["worst dress"]

seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = ['positive', 'neutral', 'negative'] 

print(pred)
print(np.argmax(pred))
print(labels[np.argmax(pred)-1])

1/1 [==============================] - 0s 27ms/step
[[6.4201909e-06 5.9856033e-01 3.5688344e-01 4.4549726e-02]]
1
positive
